In [1]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv("test.csv")

In [4]:
train = train.iloc[:, 1:]

In [5]:
test = test.iloc[:, 1:]

In [11]:
train.drop(['person_rn', 'contents_rn', 'contents_open_dt'], axis = 1, inplace = True)
test.drop(['person_rn', 'contents_rn', 'contents_open_dt'], axis = 1, inplace = True)

In [14]:
X = train.iloc[:, :-1]
y = train.target

target = test.copy()

In [37]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [16]:
skf = StratifiedKFold(n_splits = 25, random_state = 42, shuffle = True)

In [49]:
lgbm_pred = np.zeros(target.shape[0])
val_f1 = 0

In [50]:
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm = LGBMClassifier(random_state = 42, n_estimators = 10000)
    lgbm.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 500, verbose = 2500, eval_metric = 'f1')
    
    lgbm_val = lgbm.predict(val_x)
    fold_f1 = f1_score(val_y, lgbm_val) / 25
    
    val_f1 += fold_f1
    print(fold_f1 * 25)
    fold_pred = lgbm.predict_proba(target)[:, 1] / 25
    lgbm_pred += fold_pred

Training until validation scores don't improve for 500 rounds
[2500]	training's binary_logloss: 0.555392	valid_1's binary_logloss: 0.629325
Early stopping, best iteration is:
[3486]	training's binary_logloss: 0.530013	valid_1's binary_logloss: 0.627997
0.6566708050806991
Training until validation scores don't improve for 500 rounds
[2500]	training's binary_logloss: 0.555056	valid_1's binary_logloss: 0.631529
Early stopping, best iteration is:
[4023]	training's binary_logloss: 0.51671	valid_1's binary_logloss: 0.630161
0.6560421153385978
Training until validation scores don't improve for 500 rounds
[2500]	training's binary_logloss: 0.555394	valid_1's binary_logloss: 0.637702
Early stopping, best iteration is:
[3823]	training's binary_logloss: 0.521695	valid_1's binary_logloss: 0.636432
0.6511495073539911
Training until validation scores don't improve for 500 rounds
[2500]	training's binary_logloss: 0.554908	valid_1's binary_logloss: 0.636753
Early stopping, best iteration is:
[3676]	tra

In [51]:
val_f1

0.6523360175097594

In [27]:
cb_pred = np.zeros(target.shape[0])
val_f1 = 0

In [28]:
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    cb = CatBoostClassifier(random_state = 42, verbose = 0, iterations = 10000, eval_metric = 'F1')
    cb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 500, verbose = 2500)
    
    cb_val = cb.predict(val_x)
    fold_f1 = f1_score(val_y, cb_val) / 25
    
    val_f1 += fold_f1
    fold_pred = cb.predict_proba(target)[:, 1] / 25
    cb_pred += fold_pred

Learning rate set to 0.05353
0:	learn: 0.6125875	test: 0.6125875	test1: 0.6145226	best: 0.6145226 (0)	total: 33.4ms	remaining: 5m 33s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.6497818251
bestIteration = 1957

Shrink model to first 1958 iterations.
0.02599127300322519
Learning rate set to 0.05353
0:	learn: 0.6127896	test: 0.6127896	test1: 0.6106445	best: 0.6106445 (0)	total: 33.6ms	remaining: 5m 35s
2500:	learn: 0.6688212	test: 0.6688212	test1: 0.6425257	best: 0.6433660 (2350)	total: 1m 15s	remaining: 3m 46s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.6480179792
bestIteration = 3525

Shrink model to first 3526 iterations.
0.025920719169894323
Learning rate set to 0.05353
0:	learn: 0.6126731	test: 0.6126731	test1: 0.6134438	best: 0.6134438 (0)	total: 34.3ms	remaining: 5m 43s
2500:	learn: 0.6693097	test: 0.6693097	test1: 0.6427011	best: 0.6441948 (2253)	total: 1m 15s	remaining: 3m 46s
Stopped by overfitting detector  (500 iterations wait

2500:	learn: 0.6690307	test: 0.6690307	test1: 0.6435795	best: 0.6437121 (2488)	total: 1m 16s	remaining: 3m 50s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.6472408704
bestIteration = 3330

Shrink model to first 3331 iterations.
0.02588963481407418
Learning rate set to 0.05353
0:	learn: 0.6125774	test: 0.6125774	test1: 0.6157399	best: 0.6157399 (0)	total: 31.2ms	remaining: 5m 11s
2500:	learn: 0.6690760	test: 0.6690760	test1: 0.6475946	best: 0.6487564 (2414)	total: 1m 18s	remaining: 3m 56s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.6493383295
bestIteration = 2639

Shrink model to first 2640 iterations.
0.02597353317839017
Learning rate set to 0.05353
0:	learn: 0.6124057	test: 0.6124057	test1: 0.6193049	best: 0.6193049 (0)	total: 31.7ms	remaining: 5m 16s
2500:	learn: 0.6687506	test: 0.6687506	test1: 0.6504637	best: 0.6510058 (2351)	total: 1m 18s	remaining: 3m 56s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.65249845

In [36]:
val_f1

0.6480318828379916

In [52]:
sub_pred = [1 if p >= 0.5 else 0 for p in (cb_pred + lgbm_pred) / 2]

In [32]:
submission = pd.read_csv('sample_submission.csv')

In [55]:
submission['target'] = sub_pred

In [56]:
submission.to_csv('20211214_2nd.csv', index = False)

In [54]:
submission

,id,target
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
46399,46399,1
46400,46400,1
46401,46401,1
46402,46402,1
